In [1]:
import pandas as pd
import numpy as np
import pymssql
import yaml
from yaml import Loader

In [2]:
with open('secrets.yaml', 'r') as f:
    configs = yaml.load(f, Loader=Loader)

In [3]:

server = configs['data']['server']
user = configs['data']['user']
password = configs['data']['password']
database = configs['data']['database']

# define table strings
efficiency_table = 'dbo.EfficiencyScores'
safety_table = 'dbo.SafetyScores'
outcomes_table = 'dbo.ClinicalOutcomeScores'
community_table = 'dbo.EngagementScores'
payment_table = 'PaymentAndValueOfCareVals'

try:
    # connect to database with above credentials
    conn = pymssql.connect(server, user, password, database)
    
    # instantiate cursor
    cursor = conn.cursor()
    
    # get efficiency data
    efficiency_query = f'SELECT * FROM {efficiency_table}'
    efficiency = pd.read_sql(efficiency_query, conn, index_col='Efficiency_ID')
    
    # get safety data
    safety_query = f'SELECT * FROM {safety_table}'
    safety = pd.read_sql(safety_query, conn, index_col='Safety_ID')
    
    # get outcomes data
    outcomes_query = f'SELECT * FROM {outcomes_table}'
    outcomes = pd.read_sql(outcomes_query, conn, index_col='ClinicalOutcome_ID')
    
    # get community data
    community_query = f'SELECT * FROM {community_table}'
    community = pd.read_sql(community_query, conn, 'EngagementScore_ID')
    
    #get payment data
    payment_query = f'SELECT * FROM {payment_table}'
    payment = pd.read_sql(payment_query, conn, index_col='Payment_ID')
except Exception as e:
    print(e)
    
payment.head()

/home/charlie/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/charlie/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/charlie/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/charlie/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/conne

,Facility_ID,Payment,Lower_Estimate,Higher_Estimate,Payment_Category,Value_Of_Care_Category
Payment_ID,,,,,,
1,020018,13461.0000,11689.5000,15405.5000,-0.5,-0.5
2,030064,20773.2500,18978.2500,22704.0000,0.25,0
3,100140,18872.6667,17511.3333,20323.3333,-0.333333,0
4,100320,18230.0000,16388.5000,20246.5000,0,0
5,110071,18356.0000,15727.0000,20892.0000,0,0


In [4]:
joined = efficiency.merge(safety, on='Facility_ID', how='inner')
joined_1  = joined.merge(outcomes, on='Facility_ID', how='inner')
joined_2 = joined.merge(community, on='Facility_ID', how='inner')
final_join = payment.merge(joined_2, on='Facility_ID', how='left')

#### Payment Estimates Model

In [13]:
# ml imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import joblib

In [12]:
X = final_join[['Lower_Estimate', 'Higher_Estimate']]
y = final_join['Payment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lnrg = LinearRegression()
lnrg = lnrg.fit(X_train, y_train)
y_preds = lnrg.predict(X_test)
print(r2_score(y_test, y_preds))

0.9996244230990997


In [ ]:
estimates_model = joblib.dump(lnrg, )